In [ ]:
!pip install tensorflow==2.17.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 98.7 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: tensorboard
    Found existing i

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
print(tf.__version__)
model = tf.keras.models.load_model("lstm_model_predict.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Convert a SavedModel to TFLite
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Default TFLite ops
    tf.lite.OpsSet.SELECT_TF_OPS     # TensorFlow ops
]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)


2.17.1


Saved artifact at '/tmp/tmp27sp4g71'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 70), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 666), dtype=tf.float32, name=None)
Captures:
  134443113864128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084116768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084117120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084118880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443085724864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084108848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084110960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443085714480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443113704160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443113664352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084396352: Tenso

In [6]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("optimized_model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpdy5yxovs'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 70), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 666), dtype=tf.float32, name=None)
Captures:
  134443113864128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084116768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084117120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084118880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443085724864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084108848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084110960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443085714480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443113704160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443113664352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134443084396352: Tenso

In [7]:
# Load TFLite model
interpreter = tf.lite.Interpreter("optimized_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load tokenizer
with open("tokenizer_predict.pkl", "rb") as handle:
    tokenizer = pickle.load(handle)

# Prepare input text
input_text = "pph21"
sequence = tokenizer.texts_to_sequences([input_text])
padded_sequence = pad_sequences(sequence, maxlen=70, padding="post")
input_data = np.array(padded_sequence, dtype=np.float32)

# Perform prediction
interpreter.set_tensor((input_details[0]['index']), input_data)
interpreter.invoke()
predictions = interpreter.get_tensor(output_details[0]['index'])

# Get predicted class
predicted_class = np.argmax(predictions)
print("Predicted class:", predicted_class)


Predicted class: 36
